In [ ]:
import re
import pandas as pd
pd.options.display.max_columns = 9
pd.options.display.max_rows = 3
np = pd.np
np.norm = np.linalg.norm
from datetime import datetime, date
import json
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from sklearn.feature_extraction.text import TfidfVectorizer  # equivalent to TFIDFTransformer(CountVectorizer())
from django.db.models import Sum
from pacs.models import CampaignDetail, WorkingTransactions
import django
django.setup()
CampaignDetail.objects.count(), WorkingTransactions.objects.count()

# Django
This is how you join CampaignDetail & WorkingTransactions  
and aggregate the WorkingTransactions.amount

In [ ]:
qs = CampaignDetail.objects.annotate(net_amount=Sum('workingtransactions__amount')).values().all()
print('Net transactions: {:,}M'.format(round(sum(qs.values_list('net_amount', flat=True)) / 1e6)))

Convert a Django Queryset into a Pandas DataFrame

In [ ]:
df = pd.DataFrame.from_records(qs)
df.columns

In [ ]:
df = df[df.committee_name.astype(bool)].copy()
df

# Pandas DataFrame.join
What if you only want positive transactions?

In [ ]:
qs_pos = CampaignDetail.objects.filter(committee_name__isnull=False, workingtransactions__amount__gt=0)
qs_pos = qs_pos.annotate(pos_amount=Sum('workingtransactions__amount'))
df_pos = df.join(pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount'])
df_pos

### What if I just insert a new column with the values?

In [ ]:
df = pd.DataFrame.from_records(qs)
df = pd.DataFrame(df[df.committee_name.astype(bool)])
df['pos_amount'] = pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount']
df

# Pandas indices are tricky
Did all the rows get inserted in the right place (are the indices still alligned)


In [ ]:
df == df_pos

In [ ]:
pd.options.display.max_rows = 6
(df == df_pos).mean()

# A NaN is not equal to a NaN!
Any operation involving a NaN returns a NaN  
And NaN (like None) always evaluates to False  

In [ ]:
(df == df_pos).mean() + df.isnull().mean()

# Negative transaction amounts?

In [ ]:
qs_neg = CampaignDetail.objects.filter(workingtransactions__amount__lt=0)
qs_neg = qs_neg.annotate(neg_amount=Sum('workingtransactions__amount'))
df = df.join(pd.DataFrame.from_records(qs_neg.values('neg_amount').all())['neg_amount'])
df

In [ ]:
print('Positve transactions: {:,} M'.format(round(sum(qs_pos.values_list('pos_amount', flat=True)) / 1e6)))

In [ ]:
print('Negative transactions: {:,} M'.format(round(sum(qs_neg.values_list('neg_amount', flat=True)) / 1.e6, 2)))

In [ ]:
print('Net net transactions: {:,} M'.format(round(sum(qs.values_list('net_amount', flat=True)) / 1.e6)))

## Something's fishy in Denmark ^

In [ ]:
df.sum()

In [ ]:
print('Net amount: ${:} M'.format(round(df.sum()[['pos_amount', 'neg_amount']].sum()/1e6)))

In [ ]:
print('Volume: ${:} M'.format(round(np.abs(df.sum()[['pos_amount', 'neg_amount']]).sum()/1e6)))

# Directed graph of financial transactions
Are the payee_committee_ids the same as "filer_id"?

In [ ]:
filer_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'filer_id').all()).dropna().values.T[0])
payee_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'contributor_payee_committee_id').all()).dropna().values.T[0])
com_id = set()
len(payee_id.intersection(filer_id)) * 1. / len(filer_id)

# Good enough for Government Work
53% of payee_ids were found in the filer_id of the same Table
filer_id -> payee_id

In [ ]:
qs = WorkingTransactions.objects.filter(filer_id__isnull=False, 
                                        contributor_payee_committee_id__isnull=False,
                                        amount__gt=0)
df_trans = pd.DataFrame.from_records(qs.values().all())
_, trans = df_trans.iterrows().next()
print(trans)
print(trans.index.values)

# Lets compute a similarity matrix from positive transactions

In [ ]:
ids = [int(i) for i in payee_id.intersection(filer_id)]
id_set = set(ids)
id_str = [str(int(i)) for i in ids]
N = len(ids)
cov = pd.DataFrame(np.zeros((N, N)), index=id_str, columns=id_str)
print(cov)
for rownum, trans in df_trans.iterrows():
    fid = trans['filer_id_id']
    # print(trans.index.values)
    cid = trans['contributor_payee_committee_id']
    if fid in id_set and cid in id_set:
        print(cov[str(fid)][str(cid)])
        cov[str(fid)][str(cid)] += trans['amount']
cov
    